In [2]:
# Google Colab Notebook for Doctor Survey Targeting

# Install necessary libraries
!pip install gradio pandas scikit-learn openpyxl

import pandas as pd
import numpy as np
import gradio as gr
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = "dummy_npi_data.xlsx"  # Upload manually or mount Google Drive
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name="Dataset")

# Feature Engineering
df['Login Hour'] = df['Login Time'].dt.hour
le_region = LabelEncoder()
le_speciality = LabelEncoder()
df['Region Encoded'] = le_region.fit_transform(df['Region'])
df['Speciality Encoded'] = le_speciality.fit_transform(df['Speciality'])
df['High Engagement'] = (df['Count of Survey Attempts'] > df['Count of Survey Attempts'].median()).astype(int)

# Model Training
features = ['Login Hour', 'Usage Time (mins)', 'Region Encoded', 'Speciality Encoded']
X = df[features]
y = df['High Engagement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
gb_model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, random_state=42)
gb_model.fit(X_train, y_train)

# Define Gradio function
def predict_doctors(time_input):
    input_hour = datetime.strptime(time_input, "%H:%M").hour
    df['Engagement Probability'] = gb_model.predict_proba(df[features])[:, 1]
    filtered_doctors = df[df['Login Hour'] == input_hour]
    top_doctors = filtered_doctors.sort_values(by='Engagement Probability', ascending=False)
    top_doctors = top_doctors[['NPI', 'Speciality', 'Region', 'Engagement Probability']].head(10)
    output_file = "targeted_doctors.csv"
    top_doctors.to_csv(output_file, index=False)
    return output_file

# Gradio Web App
demo = gr.Interface(
    fn=predict_doctors,
    inputs=gr.Textbox(label="Enter Time (HH:MM)"),
    outputs=gr.File(label="Download Targeted Doctors CSV"),
    title="Doctor Survey Targeting App",
    description="Enter a time to find the best doctors to invite for a survey."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://37b7678908ec2586b4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
